This notebook tests out the helper functions (in `utils.py`) that (1) parse the APR spreadsheets for 2018-2019 data, and (2) combines the ABAG permits dataset from 2013-2017 with the APR spreadsheets from 2018 to 2019 to create a dataset of all permits over the entire time period.

In [138]:
import geopandas as gpd
import pandas as pd
from IPython.display import Markdown
from housing_elements import utils

Set up logging to print to the screen:

In [139]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [140]:
from importlib import reload
reload(utils)

<module 'housing_elements.utils' from '/Users/Salim/Desktop/housing-elements/housing_elements/utils.py'>

First, let's test out the APR spreadsheet helper function (`utils.load_apr_permits`) on the cities we have to ensure they look reasonable.

In [110]:
datasets = {}
for city in ['Berkeley', 'Mountain View', 'Oakland',  'Palo Alto', 'SanJose']:
    for year in ['2018', '2019']:
        filtered_df = utils.load_apr_permits(city, year)

        display(Markdown('# ' + city + ' ' + year))
        display(filtered_df[['Current APN', 'Street Address', '# of Units Issued Building Permits', 
                             'Unit Category (SFA,SFD,2 to 4,5+,ADU,MH)']])

INFO:housing_elements.utils:Column Prior APN+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:Column How many of the units were Extremely Low Income?+ is not all null (3 rows not null), will not drop it.
INFO:housing_elements.utils:Column Assistance Programs for Each Development (see instructions) is not all null (2 rows not null), will not drop it.
INFO:housing_elements.utils:Column Number of Demolished/Destroyed Units+ is not all null (7 rows not null), will not drop it.
INFO:housing_elements.utils:Column Demolished or Destroyed Units+ is not all null (9 rows not null), will not drop it.
INFO:housing_elements.utils:Column Demolished/Destroyed Units Owner or Renter+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:111 rows of # of Units issued Entitlements are not null or zero. Dropping the column anyway
INFO:housing_elements.utils:111 rows of Entitlement Date Approved are not null or zero. Dropping the column anyway
INF

# Berkeley 2018

,Current APN,Street Address,# of Units Issued Building Permits,"Unit Category (SFA,SFD,2 to 4,5+,ADU,MH)"
56,063 295203300,680 Cragmont Ave,1,ADU
57,057 204600803,2131 University Ave,205,5+
59,054 179202600,1316 Carleton St,1,ADU
60,054 170208900,2946 Claremont Blvd,1,ADU
61,054 178301100,1319 Blake St,1,ADU
...,...,...,...,...
163,064 424501200,123 Parkside Dr,1,ADU
164,052 156402200,2751 Prince St,1,ADU
165,056 196903000,2223 Seventh St,1,ADU
166,060 240601100,1224 Stannage Ave,1,ADU


INFO:housing_elements.utils:Column Prior APN+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:Column How many of the units were Extremely Low Income?+ is not all null (2 rows not null), will not drop it.
INFO:housing_elements.utils:Column Assistance Programs for Each Development (see instructions) is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:Column Number of Demolished/Destroyed Units+ is not all null (4 rows not null), will not drop it.
INFO:housing_elements.utils:Column Demolished or Destroyed Units+ is not all null (6 rows not null), will not drop it.
INFO:housing_elements.utils:Column Demolished/Destroyed Units Owner or Renter+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:105 rows of # of Units issued Entitlements are not null or zero. Dropping the column anyway
INFO:housing_elements.utils:105 rows of Entitlement Date Approved are not null or zero. Dropping the column anyway
INF

# Berkeley 2019

...

# SanJose 2018

,Current APN,Street Address,# of Units Issued Building Permits,"Unit Category (SFA,SFD,2 to 4,5+,ADU,MH)"
1,46701030,290 N 2ND STREET,84.0,5+
2,28432014,1690 Southwest Exp,64.0,5+
21,49142039,2938 GLEN DARBY CT,1.0,ADU
22,48416039,88 PALA AV,1.0,ADU
23,57725042,6478 SPROUL CT,1.0,ADU
...,...,...,...,...
506,37322022,1144 BRETMOOR WY (BEPM 100%) SECOND UNIT ...,1.0,ADU
509,42935054,1236 MINNESOTA AV (BEMP 100%) DETACHED SECOND...,1.0,ADU
519,67628001,2654 PEARTREE LN (BEPM 100%) 2ND DETACH UNIT ...,1.0,ADU
520,43960014,"1428 GERHARDT AV (BEPM 100%) 2ND UNIT, ATTACH...",1.0,ADU


INFO:housing_elements.utils:Column Prior APN+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:Column Local Jurisdiction Tracking ID+ is not all null (827 rows not null), will not drop it.
INFO:housing_elements.utils:Column How many of the units were Extremely Low Income?+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:Column Infill Units? Y/N+ is not all null (8 rows not null), will not drop it.
INFO:housing_elements.utils:Column Assistance Programs for Each Development (see instructions) is not all null (7 rows not null), will not drop it.
INFO:housing_elements.utils:Column Number of Demolished/Destroyed Units+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:Column Demolished or Destroyed Units+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:Column Demolished/Destroyed Units Owner or Renter+ is not all null (1 rows not null), will not drop it.


# SanJose 2019

,Current APN,Street Address,# of Units Issued Building Permits,"Unit Category (SFA,SFD,2 to 4,5+,ADU,MH)"
22,49910014,2772 EULALIE DR,1,ADU
23,64904012,3233 ANDORA DR,1,ADU
24,49908016,1235 BECKET DR,1,ADU
25,24104044,1599 BERRYESSA RD,233,5+
26,NaN,2727 GOMES DR,1,SFD
...,...,...,...,...
829,40338003,5272 YORKTON WAY,1,ADU
830,40330056,12730 LANTANA AVENUE,1,ADU
831,29921010,1015 PHELPS AVENUE,1,ADU
832,67318042,3210 PUMPHERSTON WAY,1,ADU


These look reasonable.

Now let's test out the functions that load the ABAG permits:

In [11]:
sj_permits_df = utils.load_all_new_building_permits('San Jose')

INFO:housing_elements.utils:Column Prior APN+ is not all null (2 rows not null), will not drop it.
INFO:housing_elements.utils:Column Local Jurisdiction Tracking ID+ is not all null (2 rows not null), will not drop it.
INFO:housing_elements.utils:Column How many of the units were Extremely Low Income?+ is not all null (3 rows not null), will not drop it.
INFO:housing_elements.utils:Column Infill Units? Y/N+ is not all null (3 rows not null), will not drop it.
INFO:housing_elements.utils:Column Assistance Programs for Each Development (see instructions) is not all null (13 rows not null), will not drop it.
INFO:housing_elements.utils:Column Number of Demolished/Destroyed Units+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:Column Demolished or Destroyed Units+ is not all null (1 rows not null), will not drop it.
INFO:housing_elements.utils:Column Demolished/Destroyed Units Owner or Renter+ is not all null (1 rows not null), will not drop it.
INFO:housi

In [12]:
sj_permits_df.groupby('permyear')['totalunit'].sum()

permyear
2014    4326.0
2015    2019.0
2016    2088.0
2017    3097.0
2018    2973.0
2019    2425.0
Name: totalunit, dtype: float64

Looks good!

In [13]:
sj_permits_df.columns

Index(['objectid', 'joinid', 'permyear', 'county', 'jurisdictn', 'apn',
       'address', 'zip', 'projname', 'hcategory', 'vlowdr', 'vlowndr',
       'vlowtot', 'lowdr', 'lowndr', 'lowtot', 'moddr', 'modndr', 'modtot',
       'amodtot', 'totalunit', 'tenure', 'mapped', 'mapnotes', 'pda',
       'pdacycle', 'pdajoinid', 'tpa', 'tpacycle', 'hsngsite', 'rhnacycle',
       'notes', 'ObjectId2', 'OBJECTID', 'geometry', 'Prior APN+',
       'Local Jurisdiction Tracking ID+', 'Building Permits Date Issued',
       'How many of the units were Extremely Low Income?+',
       'Was Project APPROVED using GC 65913.4(b)? (SB 35 Streamlining) Y/N',
       'Infill Units? Y/N+',
       'Assistance Programs for Each Development (see instructions)',
       'Deed Restriction Type (see instructions)',
       'For units affordable without financial assistance or deed restrictions, explain how the locality determined the units were affordable (see instructions)',
       'Term of Affordability or Deed Restri

In [14]:
sj_sites = utils.load_site_inventory('San Jose')

Dropping constant columns: ['rhnacyc' 'rhnayrs' 'county' 'jurisdict' 'currunits' 'allowlow' 'num_vl'
 'num_l' 'num_m' 'num_am']
DF shape (599, 18)


In [15]:
sj_permits_df.columns

Index(['objectid', 'joinid', 'permyear', 'county', 'jurisdictn', 'apn',
       'address', 'zip', 'projname', 'hcategory', 'vlowdr', 'vlowndr',
       'vlowtot', 'lowdr', 'lowndr', 'lowtot', 'moddr', 'modndr', 'modtot',
       'amodtot', 'totalunit', 'tenure', 'mapped', 'mapnotes', 'pda',
       'pdacycle', 'pdajoinid', 'tpa', 'tpacycle', 'hsngsite', 'rhnacycle',
       'notes', 'ObjectId2', 'OBJECTID', 'geometry', 'Prior APN+',
       'Local Jurisdiction Tracking ID+', 'Building Permits Date Issued',
       'How many of the units were Extremely Low Income?+',
       'Was Project APPROVED using GC 65913.4(b)? (SB 35 Streamlining) Y/N',
       'Infill Units? Y/N+',
       'Assistance Programs for Each Development (see instructions)',
       'Deed Restriction Type (see instructions)',
       'For units affordable without financial assistance or deed restrictions, explain how the locality determined the units were affordable (see instructions)',
       'Term of Affordability or Deed Restri

In [16]:
utils.calculate_ratio_of_development_on_inventory_sites(sj_sites, sj_permits_df)

Total units permitted: 16928.0
Total realistic capacity in inventory: 35536.43


0.4763562349960308

In [17]:
sj_sites[sj_sites.apn == '25417084']

,objectid,apn,locapn,genplan,zoning,gacres,locacres,allowden,allowhigh,relcapcty,desafford,sitetype,pdaparcel,existuse,localnote,Shape__Are,Shape__Len,geometry
37822,79956,25417084,25417084,UV,A(PD),35.648,12.50,20.0,20,242.0,NaN,Entitled,Y,None,NaN,0.000015,0.015521,"POLYGON ((-121.87497 37.36891, -121.87496 37.3..."
37823,79957,25417084,25417084,UV,A(PD),35.648,13.70,30.0,30,494.0,Affordable,Entitled,Y,None,NaN,0.000015,0.015521,"POLYGON ((-121.87497 37.36891, -121.87496 37.3..."
37824,79958,25417084,25417084,UV,A(PD),35.648,19.09,20.0,20,2082.0,NaN,Entitled,Y,Commercial,NaN,0.000015,0.015521,"POLYGON ((-121.87497 37.36891, -121.87496 37.3..."


In [18]:
# TODO: In the course of testing with the san jose dataset, i saw one APN listed three times.
# apn == 25417084. we should find some fix during cleaning

In [19]:
sj_sites.apn.isin(sj_permits_df.apn).sum()

64

In [20]:
sj_permits_df.apn.isin(sj_sites.apn).sum()

302

In [21]:
# TODO: why are there multiple permits per site? are some of these not used? are there duplicates?

In [22]:
utils.calculate_mean_overproduction_on_sites(sj_sites, sj_permits_df)

Number of inventory sites developed: 64
Number of units permitted on inventory sites: 3473.0
Total realistic capacity of inventory sites: 3664.0


-2.984375

In [23]:
utils.calculate_inventory_housing_over_all_housing(sj_sites, sj_permits_df)

Units permitted on inventory sites: 3473.0
Total units permitted: 16928.0


0.20516304347826086

In [24]:
utils.calculate_ratio_of_development_on_inventory_sites(sj_sites, sj_permits_df)

Total units permitted: 16928.0
Total realistic capacity in inventory: 35536.43


0.4763562349960308

In [27]:
sj_sites.sitetype.value_counts()

Underutilized           246
Vacant                  206
Entitled                103
Underutilized and Va     44
Name: sitetype, dtype: int64

In [98]:
mv_sites.allowden.value_counts()

60.0    15
25.0     5
35.0     3
6.0      2
12.0     2
50.0     2
30.0     2
4.0      1
40.0     1
38.0     1
Name: allowden, dtype: int64

In [141]:
utils.calculate_pdev_for_nonvacant_sites(sj_sites, sj_permits_df)

0.1272264631043257

In [142]:
utils.calculate_pdev_for_vacant_sites(sj_sites, sj_permits_df)

0.06796116504854369

In [143]:
utils.calculate_pdev_for_inventory(sj_sites, sj_permits_df)

0.10684474123539232